# Convolutional Model

In [ ]:
import keras
import time
from keras.models import Model
from keras.layers import Input,Dense,Conv1D,Flatten,BatchNormalization,Dropout
from keras.optimizers import RMSprop

dim = 4 # space dimension
total = 10000 #number of samples
filename = '../data/output_d'+str(dim)+'_s'+str(total)+'.csv'

## Common functions to manage the input file

In [ ]:
%run CommonFunctions.ipynb

In [ ]:
train_data, test_data = read_file_format_and_split(filename)

## Convolutional Model Configuration

In [ ]:
input_a = Input(shape=(dim,1))
input_b = Input(shape=(dim,1))

main_input = keras.layers.concatenate([input_a, input_b])

output_1 = Conv1D(100,kernel_size=(int(dim/2)),padding="same", activation='relu', input_shape=(dim,2))(main_input)
output_2 = BatchNormalization()(output_1)
output_3 = Conv1D(150,kernel_size=(int(dim/2)),padding="same", activation='relu')(output_2)
output_4 = BatchNormalization()(output_3)
output_5 = Conv1D(200,kernel_size=(int(dim/2)),padding="same", activation='relu')(output_4)
output_6 = Flatten()(output_5)
output_7 = Dropout(0.5)(output_6)
main_output = Dense(1, activation='sigmoid')(output_7)

model = Model(inputs=[input_a,input_b], outputs=main_output)
opt = RMSprop(learning_rate=0.00001)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start_time = time.time()
input_shape = (train_data.shape[0],dim,1)
input_reshaped = [np.array(list(train_data['A'])).reshape(input_shape), np.array(list(train_data['B'])).reshape(input_shape)]
model.fit(input_reshaped, np.array(list(train_data['AB'])).reshape(train_data.shape[0],1), validation_split=.2, epochs=20, verbose=2)  # starts training
train_duration = time.time() - start_time
train_duration

In [ ]:
start_time = time.time()
score = model.evaluate([np.array(list(test_data['A'])).reshape(test_data.shape[0],dim,1), np.array(list(test_data['B'])).reshape(test_data.shape[0],dim,1)], np.array(list(test_data['AB'])).reshape(test_data.shape[0],1))
test_duration = time.time() - start_time
score

In [ ]:
test_duration

## Saving model and weights

In [ ]:
config_name = "conv_100_150_200_dp05_ep20"

In [ ]:
save_model_json(model,config_name,dim,total)

In [ ]:
save_model_weights(model,config_name,dim,total)